# 👋🌍 Hello, world: Submit a Cirq job to Honeywell (Quantinuum)

In this notebook, we'll review the basics of Azure Quantum by submitting a simple *job*, or quantum program, to [Honeywell (Quantinuum)](https://www.quantinuum.com/). We will use [Cirq](https://quantumai.google/cirq) to express the quantum job.

## Submit a simple job to Honeywell using Azure Quantum
Azure Quantum provides several ways to express quantum programs. In this example, we are using Cirq, but note that Q# and Qiskit are also supported. All code in this example will be written in Python.

Let's begin. When you see a code block, hover over it and click the triangle play-button to execute it. To avoid any compilation issues, this should be done in order from top to bottom.

### 1. Connect to the Azure Quantum workspace

To connect to the Azure Quantum service, construct an instance of the `AzureQuantumService`. Note that it's imported from `azure.quantum.cirq`.

In [ ]:
from azure.quantum.cirq import AzureQuantumService

service = AzureQuantumService (
    resource_id = "",
    location = "")

Let's see what providers and targets are enabled in this workspace with the following command:


In [ ]:
targetInfo = service.targets()

print("This workspace's targets:")
for target in targetInfo:
    print("-", target.name)

### ❕ Do you see `honeywell.hqs-lt-s1-apival` in your list of targets? If so, you're ready to keep going.

Don't see it? You may need to add Honeywell to your workspace to run this sample. Navigate to the **Providers** page in the portal and click **+Add** to add the Honeywell provider. Don't worry, there's a free credits plan available.

## Honeywell (Quantinuum): The quantum provider
Azure Quantum partners with third-party companies to deliver solutions to quantum jobs. These company offerings are called *providers*. Each provider can offer multiple *targets* with different capabilities. See the table below for Honeywell's targets.

Target name | Target ID | Number of qubits | Description
| --- | ---| ---|
Quantum validator | `honeywell.hqs-lt-s1-apival`* | 12 | Honeywell's "API validator." Use this to validate quantum programs before submitting to hardware or emulation on Honeywell's platform. Free of cost. |
Honeywell System Model H1 Emulator | `honeywell.hqs-lt-s1-sim`* | 12 | Honeywell's quantum simulator. Uses a noise model that is consistent with their H1 hardware offering. |
Honeywell System Model H1 | `honeywell.hqs-lt-s1`* | 12 | Honeywell's trapped-ion quantum computer. |

\*This target contains an alternate of similar function that is referred to by changing the `s1` in its name to `s2`.

For this example, we will use `honeywell.hqs-lt-s1-apival` to avoid any costs or credit usage. If you wish to emulate or run the actual circuit, you may replace all instances of `honeywell.hqs-lt-s1-apival` in subsequent code cells with one of the other values in the table above, but please note any costs incurred. To learn more about Honeywell's targets, check out our [documentation](https://docs.microsoft.com/azure/quantum/provider-honeywell).

### 2. Build the quantum program

Let's create a simple Cirq circuit to run.

In [ ]:
import cirq

q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.H(q0),               # Apply an H-gate to q0
    cirq.measure(q0)          # Measure q0
)
circuit

The circuit you built is a simple quantum random bit generator. With Honeywell's API validator, we will be able to confirm that the circuit is able to be run on their H1 emulator and hardware.

### 3. Submit the quantum program to Honeywell

In [ ]:
# Using the Honeywell API validation target, call "run" to submit the job. We'll
# use 100 repetitions (simulated runs).
job = service.targets("honeywell.hqs-lt-s1-apival").submit(circuit, name="hello world-cirq-honeywell", repetitions=100)

# Print the job ID.
print("Job id:", job.job_id())

The job ID can be used to retrieve the results later using the `get_job` function or by viewing it under the **Job management** section of the portal.

### 4. Obtain the job results
To get the job's results, you can await completion and plot the output, as in the next code cell. This may take a few seconds.

In [ ]:
# Await job results.
result = job.results()

from matplotlib import pyplot

# Honeywell's simulator does not return probabilities. Instead, it returns a dictionary of measurements.
# (Note that since we targeted API validation, we expect to see that all measurements result in a zero.)
# In order to visualize the result, we can use a histogram. Since the circuit contains only a single
# measurement, we can expect that all values will be tied to the key "m_0".
pyplot.hist(result['m_0'], bins=[0,1,2], width=0.5)
pyplot.title("Result")
pyplot.xlabel("Measurement")
_ = pyplot.ylabel("Repititions")

**See the histogram above? Congratulations, you've submitted a job with Azure Quantum! 👏**


### 5. Next steps
Next, you can try running a program on Honeywell's emulation and hardware targets. Just replace `honeywell.hqs-lt-s1-apival` with `honeywell.hqs-lt-s1-sim` or `honeywell.hqs-lt-s1`. Or try another sample by navigating back to the sample gallery. The same "hello world" sample can be run with different quantum providers by choosing another option in the gallery card drop-down menu. Don't worry - your work here is automatically saved.

To learn more about submitting Cirq circuits to Azure Quantum, review [this documentation](https://docs.microsoft.com/azure/quantum/quickstart-microsoft-cirq?pivots=platform-honeywell).